# OPTIMIZATION PROBLEM

---
# Introduction

We have the following problem: We are looking for the optimal path from our home to our office.

Consider the figure above: we start on the top left road and want to go on the bottom right road. If we reach this objective, we earn \\$10 . We can cross the top right bridge but it will cost us \$5. We can not take the bottom left and center roads because they are closed.

<img src="Optimization_Problem.jpg" width=400>

We are tired and distracted. If we decide to move to one direction, we have 70% probability that we actually move in that direction, and 30% probability that we walk to any of the other admissible directions.

In this project, we will be considering both the deterministic and the stochastic problems.

**Recall:** Here, deterministic setting means that our decision is not affected by random noise; stochastic setting means that the 70% rule (as described above) holds when we move.

---

# Set up the notebook

In [1]:
# Run this cell to set up the notebook
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

---
# Define gamma and epsilon

In [2]:
gamma = 0.9
epsilon = 0.9

---
# PART I - DETERMINISTIC VERSION OF THE PROBLEM

First, we consider the deterministic version of the problem. By using the value iteration method, we compute the vector V(s) with s the state (taking values between 0 and 6). Then, by using such values, we compute the Q-matrix Q(s, a) with a the action (taking values between 0 and 3).

**Notice:** 
- The output of this part is one vector and one matrix. 
- For actions, we use the convention 0 = up, 1 = right, 2 = down, 3 = left.

### I. 1) Define the rewards for the deterministic version R_s_a

In [3]:
R = [
    [0,0,0,0],
    [0,-5,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [-5,0,10,0]
]

### I. 2) Define V_next

In [4]:
V_next = [
    [6,1,3,6],
    [6,2,4,0],
    [6,6,5,1],
    [0,4,6,6],
    [1,5,6,3],
    [2,6,6,4]
]

### I. 3) Bellman Equations

In [5]:
# Initialize the variables

V_ini = np.array([0,0,0,0,0,0,0])

In [6]:
# Write down the Bellman equations

V = []

for s in range(0,6):
    L = []
    for a in range(0,4):
        y = R[s][a] + gamma*V_ini[ V_next[s][a] ]
        L.append(y)
    V.append(max(L))
V.append(0)

V = np.array(V)
V    

array([ 0.,  0.,  0.,  0.,  0., 10.,  0.])

### I. 4) Iteration Method

In [7]:
# V0(s) = 0 for each s in S

V_0 = V_ini
V_0

array([0, 0, 0, 0, 0, 0, 0])

In [8]:
# First iteration V_1(s)

V_1 = V
V_1

array([ 0.,  0.,  0.,  0.,  0., 10.,  0.])

In [9]:
# Iteration

u = abs(V_0 - V_1)

while max(u) > epsilon:
    V_0 = V_1
   
    V = []
    for s in range(0,6):
        L = []
        for a in range(0,4):
            y = R[s][a] + gamma*V_0[ V_next[s][a] ]
            L.append(y)
        V.append(max(L))
    V.append(0)
    V_1 = np.array(V)
    u = abs(V_0 - V_1)

V = V_1
print('The vector {V(s)} is:')
V

The vector {V(s)} is:


array([ 7.29,  8.1 ,  9.  ,  8.1 ,  9.  , 10.  ,  0.  ])

### I. 5) Compute the Q matrix with elements Q_s_a

In [10]:
Q = []

for s in range(0,6):
    L = []
    for a in range(0,4):
        y = R[s][a] + gamma * V[ V_next[s][a] ]
        L.append(y)
    Q.append(L)
    
Q.append([0,0,0,0])

print('The Q matrix {Q(s,a)} is:')
Q

The Q matrix {Q(s,a)} is:


[[0.0, 7.29, 7.29, 0.0],
 [0.0, 3.0999999999999996, 8.1, 6.561],
 [0.0, 0.0, 9.0, 7.29],
 [6.561, 8.1, 0.0, 0.0],
 [7.29, 9.0, 0.0, 7.29],
 [3.0999999999999996, 0.0, 10.0, 8.1],
 [0, 0, 0, 0]]

---
# PART II - STOCHASTIC VERSION OF THE PROBLEM

We consider now the stochastic version of the problem. We adapt the code in part I to this setting. Again, the output is one vector and one matrix.

### II. 1) Define the probabilities for the stochastic version P_s_a_s'

In [11]:
P = [
    [[0,0,0,0,0,0,0] , [0,0.7,0,0.3,0,0,0] , [0,0.3,0,0.7,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0,0,0,0,0,0] , [0.15,0,0.7,0,0.15,0,0] , [0.15,0,0.15,0,0.7,0,0] , [0.7,0,0.15,0,0.15,0,0]],
    [[0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0.3,0,0,0,0.7,0] , [0,0.7,0,0,0,0.3,0]],
    [[0.7,0,0,0,0.30,0,0] , [0.3,0,0,0,0.7,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0.7,0,0.15,0,0.15,0] , [0,0.15,0,0.15,0,0.7,0] , [0,0,0,0,0,0,0] , [0,0.15,0,0.7,0,0.15,0]],
    [[0,0,0.7,0,0.15,0,0.15] , [0,0,0,0,0,0,0] , [0,0,0.15,0,0.15,0,0.7] , [0,0,0.15,0,0.7,0,0.15]]
]

### II. 2) Define the rewards for the deterministic version R_s_a_s'

In [12]:
Reward = [
    [[0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0,0,0,0,0,0] , [0,0,-5,0,0,0,0] , [0,0,-5,0,0,0,0] , [0,0,-5,0,0,0,0]],
    [[0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0] , [0,0,0,0,0,0,0]],
    [[0,0,-5,0,0,0,10] , [0,0,0,0,0,0,0] , [0,0,-5,0,0,0,10] , [0,0,-5,0,0,0,10]]
]

### II. 3) Define V_next

In [13]:
V_next = [
    [6,1,3,6],
    [6,2,4,0],
    [6,6,5,1],
    [0,4,6,6],
    [1,5,6,3],
    [2,6,6,4]
]

### II. 4) Bellman Equations

In [14]:
V_ini = [0,0,0,0,0,0,0,0]
V = []

for s in range(0,6):
    L = []
    for a in range(0,4):
        u = 0
        for ss in range(0,7):
            u += P[s][a][ss] * ( Reward[s][a][ss] + gamma*V_ini[ V_next[s][a] ] )
        L.append(u)
    V.append(max(L))
    
V.append(0)

print(V)       

[0.0, 0.0, 0.0, 0.0, 0.0, 6.25, 0]


### II. 5) Iteration Method

In [15]:
# V0(s) = 0 for each s in S

V_0 = np.array([0,0,0,0,0,0,0])
V_0

array([0, 0, 0, 0, 0, 0, 0])

In [16]:
# First iteration V_1(s)

V_1 = np.array(V)
V_1

array([0.  , 0.  , 0.  , 0.  , 0.  , 6.25, 0.  ])

In [17]:
# Iteration

u = abs(V_0 - V_1)

while max(u) > epsilon:
    V_0 = V_1
    
    V = []
    for s in range(0,6):
        L = []
        for a in range(0,4):
            y = 0
            for ss in range(0,7):
                y += P[s][a][ss] * ( Reward[s][a][ss] + gamma*V_0[ V_next[s][a] ] )
            L.append(y)
        V.append(max(L))
    V.append(0)
    
    V_1 = np.array(V)
    u = abs(V_0 - V_1)

V = V_1
print('The vector {V(s)} is:')
V

The vector {V(s)} is:


array([4.55625, 4.3125 , 5.625  , 5.0625 , 5.625  , 6.25   , 0.     ])

### II. 6) Compute the Q matrix with elements Q_s_a

In [18]:
Q = []

for s in range(0,6):
    L = []
    for a in range(0,4):
        y = 0
        for ss in range(0,7):
            y+= P[s][a][ss] * (Reward[s][a][ss] + gamma * V[ V_next[s][a] ])
        L.append(y)
    Q.append(L)
    
Q.append([0,0,0,0])
Q

[[0.0, 3.8812499999999996, 4.55625, 0.0],
 [0.0, 1.5625, 4.3125, 3.350625000000001],
 [0.0, 0.0, 5.625, 3.8812499999999996],
 [4.100625000000001, 5.0625, 0.0, 0.0],
 [3.8812499999999996, 5.625, 0.0, 4.55625],
 [3.0625, 0.0, 6.25, 5.8125],
 [0, 0, 0, 0]]

---
# CONCLUSIONS

V represents the optimal strategy to adopt to solve the problem. 

In the first case, the problem is deterministic. It means that the reward and V only depend on the current state s and on the action a since the destination is deterministic and then known. Therefore, it makes sense that the V and then Q in the first case are higher than the V and Q in the second case. Thus, the values in Q in the stochastic problem decrease. However, even if the values decrease, we can see that the maximum in each row stay in the same position: the optimal strategy is still the same for the two problems!

Optimal strategy to go to Etcheverry Hall for the stochastic problem:

- 0 – 3 – 4 – 5 – 6


Finally, we can see that for the deterministic problem, there are two different optimal strategies since the probabilities are not taken into account:

- 0 – 3 – 4 – 5 – 6
- 0 – 1 – 4 – 5 – 6